# 全国大学生数学建模大赛2024年 C题 农作物的种植策略

## 问题一：
### 假定各种农作物未来的预期销售量、种植成本、亩产量和销售价格相对于2023年保持稳定，每季种植的农作物在当季销售。如果某种作物每季的总产量超过相应的预期销售量，超过部分不能正常销售。请针对以下两种情况，分别给出该乡村2024~2030年农作物的最优种植方案，将结果分别填入result1_1.xlsx和result1_2.xlsx中（模板文件见附件3）。

### (1) 超过部分滞销，造成浪费；(2) 超过部分按2023年销售价格的50%降价出售。